In [1]:
import numpy as np
import pandas as pd
import random
np.random.seed(0)
random.seed(0)
import pandas as pd
from imblearn.over_sampling import SMOTE # unbalanced dataset을 학습시키기 위해, smote 사용하여 oversampling(train set만)
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
import os

os.chdir('../data')

# xgboost 모델 복원

In [2]:
#preprocessing
data25 = pd.read_csv("feature25.csv")
data50 = pd.read_csv("feature50.csv")

data25['is_bump40'] = data25['shortest_bump_dist'] < 40
data50['is_bump40'] = data50['shortest_bump_dist'] < 40

data25['is_sidewalk25'] = data25['shortest_sidewalk_dist'] < 25
data50['is_sidewalk25'] = data50['shortest_sidewalk_dist'] < 25

col = {"x_common": ['overspeed_cam_count100', 'floating_pop_count50','car_count1000', 'child_count1000', 
                  'elem_kinder_count400','num_cram_school_count400', 
                  'shortest_cross_dist','shortest_traffic_signal_dist',],
       "x_25" : ['is_bump40', 'parking_count25','is_sidewalk25', 'barrier_nearby_count25',
                   'chaos1_nearby_count25', 'width_nearby_count25','cross_road_nearby_count25',
                   ],
       "x_50" : ['is_bump40', 'parking_count25','is_sidewalk25', 'barrier_nearby_count25',
                   'chaos1_nearby_count25', 'width_nearby_count25','cross_road_nearby_count25'],
       "y_25" : 'accident_count12.5',
       "y_50" : 'accident_count25'}

xy50 = data50[col["x_common"] + col["x_50"] +[col['y_50']]].astype(float)
xy25 = data25[col["x_common"] + col["x_25"] +[col['y_25']]].astype(float)

xy50[col['y_50']] = (xy50[col['y_50']] > 0).astype(float)
xy25[col['y_25']] = (xy25[col['y_25']] > 0).astype(float)

kmeans50 = pd.read_csv("kmeans_cluster50.csv")
kmeans25 = pd.read_csv("kmeans_cluster25.csv")

xy50=pd.concat([xy50, kmeans50], axis=1)
xy50.rename(columns={'0':'cluster'}, inplace=True)

xy25=pd.concat([xy25, kmeans25], axis=1)
xy25.rename(columns={'0':'cluster'}, inplace=True)

xy_train=xy50.drop(['overspeed_cam_count100', 'is_bump40'], axis=1)
xy_test=xy25.drop(['overspeed_cam_count100', 'is_bump40'], axis=1)

xy_train.rename(columns={'parking_count25':'parking_count', 'is_sidewalk25':'is_sidewalk',
                    'barrier_nearby_count25':'barrier_nearby_count', 'chaos1_nearby_count25':'chaos1_nearby_count',
                     'width_nearby_count25':'width_nearby_count', 'cross_road_nearby_count25':'cross_road_nearby_count',
                     'accident_count25':'accident_count'},inplace=True)
xy_test.rename(columns={'parking_count25':'parking_count', 'is_sidewalk25':'is_sidewalk',
                    'barrier_nearby_count25':'barrier_nearby_count', 'chaos1_nearby_count25':'chaos1_nearby_count',
                     'width_nearby_count25':'width_nearby_count', 'cross_road_nearby_count25':'cross_road_nearby_count',
                     'accident_count12.5':'accident_count'},inplace=True)
xy_train['cluster']=xy_train['cluster'].astype('category')
xy_test['cluster']=xy_test['cluster'].astype('category')

xy_train=pd.get_dummies(xy_train)
xy_test=pd.get_dummies(xy_test)

X_train=xy_train.drop(['accident_count'], axis=1)
y_train=xy_train['accident_count']

X_test=xy_test.drop(['accident_count'], axis=1)
y_test=xy_test['accident_count']

sm = SMOTE(random_state = 0)
X_resampled, y_resampled = sm.fit_sample(X_train, y_train)

params={'colsample_bytree': 0.5182219992643773,
 'gamma': 0.04205796920017294,
 'learning_rate': 0.0924126523646669,
 'max_depth': 10,
 'min_child_weight': 2,
 'n_estimators': 1543,
 'num_leaves': 22,
 'subsample': 0.6101047022558219}

xgb_model=XGBClassifier(params=params,random_state = 0)
xgb_model.fit(X_resampled, y_resampled)

c:\users\ljw56\anaconda3\envs\geo\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:51:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { params } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:51:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              params={'colsample_bytree': 0.5182219992643773,
                      'gamma': 0.04205796920017294,
                      'learning_rate': 0.0924126523646669, 'max_depth': 10,
                      'min_child_weight': 2, 'n_estimators': 1543,
                      'num_leaves': 22, 'subsample': 0.6101047022558219},
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

모델 복원여부 확인

In [3]:
xgb_pred_prob = xgb_model.predict_proba(X_test)
(pd.read_csv("modern_opt_set_BO.csv").iloc[:,2:]["danger"] - xgb_pred_prob[:,1]).abs().round(2).value_counts()

0.0    7636
Name: danger, dtype: int64

## Lime으로 변수별 영향 파악

In [4]:
import lime
import lime.lime_tabular
from matplotlib import font_manager, rc, rcParams
import matplotlib.pyplot as plt

In [5]:
opt_set = \
pd.concat([pd.read_csv("result_danger_loc(school).csv",index_col = 0)[["lat","lon"]],
pd.read_csv("result_danger_loc(non-school).csv",index_col = 0)[["lat","lon"]]],axis=0).reset_index()

In [6]:
def save_feature_fig(opt_set, num):
    i = opt_set.loc[num,"index"]
    explainer = lime.lime_tabular.LimeTabularExplainer(training_data=X_test.values, mode='classification',feature_names=X_test.columns, 
                                                       categorical_features=[i for i in X_test.columns if "cluster" in i or "is_" in i], 
                                                       verbose=True)

    exp = explainer.explain_instance(X_test.values[i], xgb_model.predict_proba, num_features=10)
    # exp.show_in_notebook(show_table=True,show_all=True)
    exp.as_pyplot_figure(); # 세미콜론 붙이니까 1개나오네(안 붙이면 그래프 2개 나옴)
    # plt.yticks(color='white')
    # plt.xticks(color='white')
    plt.tight_layout()
    name = str(num) + ": " + str(i) + ", lat" +  str(opt_set.loc[num,"lat"]) + ", lon" + str(opt_set.loc[num,"lon"])
    plt.title( name)
    # plt.show()

    plt.savefig('picture/'+ name.replace(": ","_") + '.png')
    plt.close()
    

In [7]:
for i in range(len(opt_set)):
    save_feature_fig(opt_set, i)

Intercept 0.10406919611731863
Prediction_local [0.58657475]
Right: 0.9969896
Intercept 0.07158842725610048
Prediction_local [0.44134121]
Right: 0.99687386
Intercept 0.23663537501662732
Prediction_local [0.4782617]
Right: 0.9945585
Intercept 0.18456521698404782
Prediction_local [0.50709443]
Right: 0.9935588
Intercept 0.19470882469633638
Prediction_local [0.51628927]
Right: 0.9901434
Intercept 0.14498852975914306
Prediction_local [0.4212383]
Right: 0.98963296
Intercept 0.30438884969917157
Prediction_local [0.32420625]
Right: 0.9876024
Intercept 0.31077153161808335
Prediction_local [0.27370253]
Right: 0.9864451
Intercept 0.15543811876053462
Prediction_local [0.39187578]
Right: 0.98606145
Intercept 0.08040273302474124
Prediction_local [0.71804311]
Right: 0.9850012
Intercept 0.0992303052317097
Prediction_local [0.5270071]
Right: 0.9843779
Intercept 0.19196153522977755
Prediction_local [0.4518234]
Right: 0.9823883
Intercept 0.26367431145693987
Prediction_local [0.38062933]
Right: 0.9784273
I

# classic Model Beta

In [8]:
tmp = pd.read_csv("classic_beta.csv",index_col = 0)

base_coef = tmp[tmp.hue == "base"][["x","effect"]]
cluster3_coef = tmp[tmp.hue == "cluster3"][["x","effect"]]
cluster0_coef = tmp[tmp.hue == "cluster0"][["x","effect"]]
schoolzone_coef = tmp[tmp.hue == "schoolzone"][["x","effect"]]

cluster25 = pd.read_csv("kmeans_cluster25.csv")

In [9]:
data25 = pd.read_csv("feature25.csv")
data50 = pd.read_csv("feature50.csv")

data25['is_bump20'] = data25['shortest_bump_dist'] < 20
data25['is_bump40'] = data25['shortest_bump_dist'] < 40
data50['is_bump20'] = data50['shortest_bump_dist'] < 20
data50['is_bump40'] = data50['shortest_bump_dist'] < 40

data25['is_sidewalk12.5'] = data25['shortest_sidewalk_dist'] < 12.5
data25['is_sidewalk25'] = data25['shortest_sidewalk_dist'] < 25
data50['is_sidewalk12.5'] = data50['shortest_sidewalk_dist'] < 12.5
data50['is_sidewalk25'] = data50['shortest_sidewalk_dist'] < 25

col = {"x_common": ['overspeed_cam_count100', 'floating_pop_count50','car_count1000', 'child_count1000', 
                  'elem_kinder_count400','num_cram_school_count400', 
                  'shortest_cross_dist','shortest_traffic_signal_dist',], # 'numberSchoolZone_count400'
       "x_25" : ['is_bump20', 'parking_count12.5','is_sidewalk12.5', 'barrier_nearby_count12.5',
                   'chaos1_nearby_count12.5', 'width_nearby_count12.5','cross_road_nearby_count12.5',
                   ],
       "x_50" : ['is_bump40', 'parking_count25','is_sidewalk25', 'barrier_nearby_count25',
                   'chaos1_nearby_count25', 'width_nearby_count25','cross_road_nearby_count25'],
       "y_25" : 'accident_count12.5',
       "y_50" : 'accident_count25',}

tmp_data50 = data25[['floating_pop_count50', 'elem_kinder_count400','num_cram_school_count400', 
                 'parking_count25', 'width_nearby_count25','cross_road_nearby_count25',]].astype(float)

tmp_data25 = data25[['floating_pop_count50', 'elem_kinder_count400','num_cram_school_count400', 
                 'parking_count25', 'width_nearby_count25','cross_road_nearby_count25',]].astype(float)
scaler = MinMaxScaler()
scaler.fit(tmp_data50)
tmp_data25.loc[:,:] = scaler.transform(tmp_data25)
data25["is_cross50"] = data25["shortest_cross_dist"] < 50
data25["is_traffic_signal50"] = data25["shortest_traffic_signal_dist"] < 50

base_coef_mean = base_coef.set_index("x")
cluster3_coef_mean = cluster3_coef.set_index("x")
cluster0_coef_mean = cluster0_coef.set_index("x")
schoolzone_coef_mean = schoolzone_coef.set_index("x")

In [10]:
dataset = pd.concat([tmp_data25,data25[['lat','lon','overspeed_cam_count100', "is_cross50", 
                                        "is_traffic_signal50",'is_bump40','numberSchoolZone_count400']]],axis=1)
dataset = dataset.assign(cluster = cluster25)
def cal_danger(x):
    coef_set = [base_coef_mean]
    if x["cluster"] == 3:
        coef_set.append(cluster3_coef_mean)
    if x["cluster"] == 0:
        coef_set.append(cluster0_coef_mean)
    if x['numberSchoolZone_count400'] != 0:
        coef_set.append(schoolzone_coef_mean)
    coef = pd.concat(coef_set,axis=1).mean(axis = 1)
    return (coef * x[coef.index])
effect_size_set = dataset.apply(cal_danger,axis=1)

In [11]:
opt_set = \
pd.concat([pd.read_csv("result_danger_loc(school).csv",index_col = 0)[["lat","lon"]],
pd.read_csv("result_danger_loc(non-school).csv",index_col = 0)[["lat","lon"]]],axis=0).reset_index()

In [12]:
def save_fig_classic(num, opt_set, effect_size_set):
    i = opt_set.loc[num,"index"]
    tmp = effect_size_set.loc[i].reset_index()
    tmp["x"] = tmp["x"].apply(lambda x: "".join([i for i in x if not i.isdigit()]))
    fig,ax = plt.subplots(figsize = (10,3))
    fig.autofmt_xdate(rotation=10)
    name = str(num) + ": " + str(i) + ", lat" +  str(opt_set.loc[num,"lat"]) + ", lon" + str(opt_set.loc[num,"lon"])
    plt.title(name)
    sns.barplot(x = tmp["x"], y = tmp.iloc[:,-1],ax=ax)
    plt.savefig('picture/'+ name.replace(": ","_") + '_classic.png')    
    plt.close()
    

In [13]:
for i in range(len(opt_set)):
    save_fig_classic(i, opt_set, effect_size_set)